# fix_pdb

> Fix PDB files
>
> 1. Pad all lines to have 80 characters (empty TER lines cause parsing issues in Biopython)
> 2. Optionally: avoid relying on segment identifiers (not used by Biopython), and give chains coming from different segments a unique identifier
> 3. Optionally: run commands from the *pdb-tools* suite

In [ ]:
#| default_exp fix_pdb

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import shutil
from pathlib import Path
import subprocess
from collections import defaultdict

import numpy as np

from fastcore.script import *

In [ ]:
#| export
def _is_ter_or_remark(pdb_line: str):
    return pdb_line.startswith("TER") or pdb_line.startswith("REMARK")


def _get_chain_and_segment_id(pdb_line: str):
    return [pdb_line[21], pdb_line[75:77].rstrip()]


@call_parse
def main(
    original_structures_dir: Param("Directory containing original structures", str),
    fixed_structures_dir: Param("Output directory containing fixed structures", str),
    rename_chains: Param(
        "Rename chains if chains from different segments have the same identifiers",
        store_true,
        default=False,
    ),
    pdb_sort: Param(
        "Run `pdb_sort` from pdb-tools if installed", store_true, default=False
    ),
    pdb_tidy: Param(
        "Run `pdb_tidy` from pdb-tools if installed", store_true, default=False
    ),
    pdb_reatom: Param(
        "Run `pdb_reatom` from pdb-tools if installed", store_true, default=False
    ),
) -> dict:
    original_structures_dir = Path(original_structures_dir)
    fixed_structures_dir = Path(fixed_structures_dir)
    fixed_structures_dir.mkdir(parents=True, exist_ok=True)

    old_to_new_chain_segment_pairs = defaultdict()

    original_structures_paths = {
        x.name.removesuffix(".pdb"): x
        for x in sorted(original_structures_dir.glob("*.pdb"))
    }

    for path in original_structures_paths.values():
        path_to_fixed_copy = fixed_structures_dir / path.name
        shutil.copy(path, path_to_fixed_copy)

        # Pad all lines to have 80 characters
        with open(path_to_fixed_copy, "r") as f:
            stripped_lines = [l.rstrip() for l in f]
            with open(path_to_fixed_copy, "w") as g:
                text = "\n".join([f"{l}{' ' * (80 - len(l))}" for l in stripped_lines])
                g.write(text)

    if rename_chains:
        for name, path in original_structures_paths.items():
            # Rename chains so that no two chains from different segments have the same identifier
            path_to_fixed_copy = fixed_structures_dir / path.name
            with open(path_to_fixed_copy, "r") as f:
                chains_and_segments = np.array(
                    [
                        _get_chain_and_segment_id(line)
                        for line in f
                        if not _is_ter_or_remark(line)
                    ]
                )
                unique_chain_segment_pairs = np.unique(chains_and_segments, axis=0)
                unique_chains = np.unique(chains_and_segments[:, 0])
                if len(unique_chains) < len(unique_chain_segment_pairs):
                    old_to_new = {}
                    for i, (chain, segment) in enumerate(unique_chain_segment_pairs):
                        old_to_new[(chain, segment)] = string.ascii_uppercase[i]
                    with open(path_to_fixed_copy, "r") as f:
                        text = ""
                        for line in f:
                            if not _is_ter_or_remark(line):
                                chain_id, seg_id = _get_chain_and_segment_id(line)
                                new_chain_id = old_to_new[(chain_id, seg_id)]
                                new_line = f"{line[:21]}{new_chain_id}{line[22:]}"
                                text += new_line
                            else:
                                text += line
                    old_to_new_chain_segment_pairs[name] = old_to_new
                    with open(path_to_fixed_copy, "w") as g:
                        g.write(text)

    print(
        f"Fixed {len(original_structures_paths)} structures, wrote outputs to {fixed_structures_dir}."
    )

    print("Modified chain IDs ((chain ID, segment ID) -> new chain ID):")
    for key, value in old_to_new_chain_segment_pairs.items():
        print(f"  - {key}: {value}")

    if pdb_sort:
        for path in original_structures_paths.values():
            path_to_fixed_copy = fixed_structures_dir / path.name
            output = subprocess.run(
                ["pdb_sort", path_to_fixed_copy], capture_output=True, text=True
            )
            with open(path_to_fixed_copy, "w") as f:
                f.write(output.stdout)

    if pdb_tidy:
        for path in original_structures_paths.values():
            path_to_fixed_copy = fixed_structures_dir / path.name
            output = subprocess.run(
                ["pdb_tidy", "-strict", path_to_fixed_copy],
                capture_output=True,
                text=True,
            )
            with open(path_to_fixed_copy, "w") as f:
                f.write(output.stdout)

    if pdb_reatom:
        for path in original_structures_paths.values():
            path_to_fixed_copy = fixed_structures_dir / path.name
            output = subprocess.run(
                ["pdb_reatom", path_to_fixed_copy], capture_output=True, text=True
            )
            with open(path_to_fixed_copy, "w") as f:
                f.write(output.stdout)

    return old_to_new_chain_segment_pairs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()